In [1]:
%run stdPackages.ipynb
os.chdir(d['py'])
from modelData import *
from gmsPython import nestingTree

No clean-up of work-folder


# Model data for SPRCG

Static Producers, Ramsey Consumers, Government.

#### Load data

In [2]:
t0 = 2019

*This creates a copy of the IO data from the ```IOdata``` project into the local data folder*

In [3]:
ioName = f'IO{t0}_OTP_Simple'
f_IOdata = os.path.join(d['project'], 'IOdata','data','processedData', ioName)
shutil.copy(f_IOdata, os.path.join(d['data'], ioName))

'C:\\Users\\sxj477\\Documents\\GitHub\\CGE_Generator\\projects\\OptimalTaxPolicy\\data\\IO2019_OTP_Simple'

*Define main settings:*

In [4]:
T  = t0 + 10 # set finite time horizon
name = 'vSPRCG' # Some global name that carries through all models/data to identify this version

*Load data:*

In [5]:
db = GpyDB(os.path.join(d['data'], ioName), name = f'{name}_{t0}', ws = d['work'])

*For relevant variables, add yearly index to data (this is currently all of them)*

In [6]:
def add_t0(k):
    k.index = stdSort(pd.MultiIndex.from_frame(k.index.to_frame(index=False).assign(t = t0)))
[add_t0(db(k)) for k in db.getTypes(['var','par'])];

*Add various time subsets that we'll rely on:*

In [7]:
addTimeToDB(t0, T, db)

#### Remove sectors and taxes:

Remove trade and inventory:

In [8]:
c = ('not', ('or', [db('s_f'), pd.Index(['itory'], name = 's'), db('n_F')]))
db['qD'] = adj.rc_pd(db('qD'), c)
db['d_qD'] = adj.rc_pd(db('d_qD'), c)

Set zero taxes:

In [9]:
db['TotalTax'] = adj.rc_pd(pd.Series(0, index = db('TotalTax').index), c, pm = False)
db['vTax']     = adj.rc_pd(pd.Series(0, index = db('vTax').index), c, pm = False)

#### Remove durables:

*We empty the relevant subsets related to durables, because we want to rely on the code that may feature durables. The empty subsets will mean that relevant equations are simply inactive.*

In [10]:
db['qD'] = adj.rc_pd(db('qD'), ('not', db('dur_p')))
[db.series.__delitem__(k) for k in ('pD_dur', 'vD_inv','vD_dur','vD_depr') if k in db.symbols];
db['dur2inv'] = adj.rc_pd(db('dur2inv'), ('not', db('dur_p')))
db['dur_p'] = adj.rc_pd(db('dur_p'), ('not', db('dur_p')))

#### Balance IO:

*Adjust house supply of labor to match demand + government spending (implicit assumption: government spending financed by lump-sum transfers from households)*

In [11]:
db.aom(pd.Series((db('qD').xs((t0, 'HH'), level = ['t','s']).sum()
                 +db('qD').xs((t0, 'G'), level = ['t','s']).sum()),
                 index = adj.rc_pd(db('qS'), db('s_HH')).index),
       name = 'qS')

*Adjust intermediate goods supply to match demand:*

In [12]:
db.aom(pd.Series(adj.rc_pd(db('qD'), db('s_i')).sum(),
                 index = adj.rc_pd(db('qS'), db('s_i')).index),
       name = 'qS')

*Adjust production sector's demand for labor and intermediate goods:*

In [13]:
db.aom(stdSort(pd.Series(0, index = adj.rc_pd(db('qD'), ('and', [db('s_p'), ('not', db('n_p'))])).index)
        +adj.rc_pd(db('qS').droplevel('s'), ('not', db('n_p')))
        ), name  = 'qD')

*Adjust production sector's own consumption to match its total supply:*

In [14]:
db.aom(adj.rc_pd(db('qS'), db('s_p'))-adj.rc_pd(db('qD'), ('and', [db('n_p'), ('not', db('s_p'))])).sum(),
       name = 'qD')

## 1. Production module

Add simple nesting:

In [15]:
m_ = pd.MultiIndex.from_tuples([('DomProdSec', 'I_K'),
                                ('DomProdSec', 'L'),
                                ('DomProdSec', 'DomProdSec_input')], names = ['n','nn'])
m = pyDatabases.cartesianProductIndex([db('s_p'), m_])
db['nestProduction'] = m

Elasticities at the same all around:

In [16]:
db.aom(pd.Series(0.5, index = m.droplevel('nn').unique()), name = 'sigma')

Set up simple empty structures for the emission/regulation module:

In [17]:
output = adj.rc_pd(db['qS'], db['s_p']) # output
inputs = adj.rc_pd(db['qD'], ('and', [db['s_p'], ('not', db['dur_p'])])) # inputs
outShares = output/pyDatabases.pdSum(output, 'n') # output shares
db['qCO2'] = (db('qCO2') * outShares).dropna() # overwrite qCO2 to be split into potentially multiple outputs from each sector

Tax rates:

In [18]:
db['tauCO2'] = adjMultiIndex.applyMult((db('vTax').xs('Emissions',level='taxTypes') / db('qCO2').replace(0,1)).fillna(0), output.index)
db['dtauCO2']= db('tauCO2').index.droplevel('t').unique()
db['dqCO2']  = db('qCO2').index.droplevel('t').unique()
db.aom(adj.rc_pd(db('TotalTax'), db['s_p'])-db('vTax').xs('Emissions',level='taxTypes'), name = 'tauLump')
db.aom(pd.Series(0, index = output.index), name = 'tauS')
db.aom(pd.Series(0, index = inputs.index), name = 'tauD')

Define sector-specific prices given regulation:

In [19]:
db.aom(stdSort(adj.rc_pd(((1+db('tauD'))*db('p')).dropna(), inputs)), name = 'pD')

## 2. Investment module

*Nesting*

In [20]:
m_ = pd.MultiIndex.from_tuples([('I_K', 'DomProdSec')], names = ['n','nn'])
m = pyDatabases.cartesianProductIndex([db('s_i'), m_])
db['nestInvestment'] = m

*Elasticity:*

In [21]:
db.aom(pd.Series(.5, index = m.droplevel('nn').unique()), name = 'sigma')

*Tax rates:*

In [22]:
output = adj.rc_pd(db['qS'], db['s_i']) # output
inputs = adj.rc_pd(db['qD'], db['s_i']) # inputs
db.aom(adj.rc_pd(db('TotalTax'), db['s_i']), name = 'tauLump')
db.aom(pd.Series(0, index = output.index), name = 'tauS')
db.aom(pd.Series(0, index = inputs.index), name = 'tauD')
db.aom(stdSort(adj.rc_pd(db('tauD')+db('p'), inputs)), name = 'pD')

## 3. Households

In [23]:
m_ = pd.MultiIndex.from_tuples([('C_HH', 'DomProdSec')], names = ['n','nn'])
m = pyDatabases.cartesianProductIndex([db('s_HH'), m_])
db['nestHH'] = m

Add mapping from consumption aggregate to labor:

In [24]:
db['L2C'] = pd.MultiIndex.from_arrays([db('s_HH'), pd.Index(['L']*len(db('s_HH')), name = 'n'), ('C_'+db('s_HH')).rename('nn')])

Elasticities:

In [25]:
db.aom(pd.Series(.5, index = m.droplevel('nn').unique()), name = 'sigma')
db.aom(pd.Series(0.25, index = db('s_HH')), name = 'frisch')

*Add lump-sum tax used to finance government consumption:*

In [26]:
output = adj.rc_pd(db('qS'), db['s_HH'])
inputs = adj.rc_pd(db('qD'), db['s_HH'])
db.aom(pd.Series(0, index = inputs.index), name = 'tauD')
db.aom(pd.Series(0, index = output.index), name = 'tauS')
db.aom(adj.rc_pd(db('qS'), db('s_HH')).groupby(['t','s']).sum()-adj.rc_pd(db('qD'), db('s_HH')).groupby(['t','s']).sum(), name = 'tauLump')
db.aom(adj.rc_pd(db('tauLump'), db('s_HH')).copy(), name = 'TotalTax')

Define sector-specific prices given regulation:

In [27]:
db.aom(stdSort(adj.rc_pd((1+db('tauD'))*db('p'), inputs)), name = 'pD')
db.aom(stdSort(adj.rc_pd(db('p')*(1-db('tauS')), output)), name = 'pS') # in this case pS is the after-tax wage rate

Specify that the household starts with zero wealth:

In [28]:
# db.aom(pd.Series(0, index = adj.rc_pd(db('tauLump'), db('s_HH')).index), name = 'vA')
db.aom(pd.Series(1, index = adj.rc_pd(db('tauLump'), db('s_HH')).index), name = 'vA') # start with a small level of assets

## 4. Government

*Nesting:*

In [29]:
m_ = pd.MultiIndex.from_tuples([('GC_G', 'DomProdSec')], names = ['n','nn'])
# m_ = pd.MultiIndex.from_tuples([], names = ['n','nn'])
m = pyDatabases.cartesianProductIndex([db('s_G'), m_])
db['nestG'] = m

*Elasticity:*

In [30]:
db.aom(pd.Series(.5, index = m.droplevel('nn').unique()), name = 'sigma')

*Adhoc regulation:*

In [31]:
inputs = adj.rc_pd(db('qD'), db('s_G'))
db['TotalTax'] = adj.rc_pd(db('TotalTax'), ('not', pd.Index(['itory'],name='s'))) # remove inventory taxes
db['d_TotalTax'] = db['TotalTax'].index.droplevel('t').unique() # what sectors pay taxes
db.aom(pd.Series(0, index = inputs.index), name = 'tauD')
db.aom(stdSort(adj.rc_pd(db('tauD')+db('p'), inputs)), name = 'pD')

## 5. Emissions

We apply the EOP abatement module, but with no abatement potential. The following creates a lot of empty indices/variables required to initialize the module:

Create abatement technology toy data. This creates a number of CCS-like technologies with costs decreasing over time at a rate $g$ to some year (here 2050):

In [32]:
techIdx = pd.Index([], name = 'tech')
techMIdx = pd.MultiIndex.from_tuples([], names = ['tech','t'])
db['techCost'] = pd.Series([], index = techMIdx)
db['techPot'] = pd.Series([], index = techMIdx)
db['DACCost'] = pd.Series(0, index = db('t'))
db['tech'] = techIdx
db['qCO2Base'] = 0

db['uCO2'] = adj.rc_pd(db('qCO2'), db('s_p')) / adj.rc_pd(db('qS'), db('s_p')) # co2 share
db['tauCO2agg'] = (db('tauCO2') * db('qCO2')).groupby('t').sum() / (db('qCO2').groupby('t').sum()) # average CO2 tax
db['tauDist'] = pd.Series(1, index = db('tauCO2').index)
db['qCO2agg'] = db('qCO2').groupby('t').sum()
db['tauEffCO2'] = db('tauCO2').copy()

Add technology smoothing parameters (adhoc levels for now):

In [33]:
db.aom(pd.Series(1, index = db('t')), name = 'DACSmooth')
db.aom(pd.Series([], index = db('techPot').index), name = 'techSmooth')

## 6. Globals

Other parameters/variables used throughout:

In [34]:
db.aom(pd.Series(db('R_LR'), index = db('t')), name = 'Rrate', priority = 'first') # fix interest rate path at long run level
db.aom(pd.Series(db('R_LR')*(1+db('infl_LR')), index = db('t')), name = 'iRate', priority = 'first')

## Export

In [35]:
db.export(repo=d['data'])